In [3]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from skimage.feature import local_binary_pattern
from skimage.feature import graycomatrix, graycoprops
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob

# Download Images



In [4]:
#Downlaod images from drive/MyDrive/CS419/Outex_TC_00012

"""
source_dir = '/content/drive/MyDrive/CS419/Assignment4/Outex_TC_00012'
images_dir = '/content/drive/MyDrive/CS419/Assignment4/Outex_TC_00012/images'

"""

images_dir = './imageset/images/'


train_0 = []
test_0 = []
test_1 = []

filenames = os.listdir(images_dir)

#sort filenames to correctly match the labels
filenames.sort()
#print(filenames)

#read images
for i in range(480):
  image = cv2.imread(os.path.join(images_dir, filenames[i]), cv2.IMREAD_GRAYSCALE)
  train_0.append(image)

for i in range(480, 4800):
  test_0.append(cv2.imread(os.path.join(images_dir, filenames[i]), cv2.IMREAD_GRAYSCALE))

for i in range(4800, len(filenames)):
  test_1.append(cv2.imread(os.path.join(images_dir, filenames[i]), cv2.IMREAD_GRAYSCALE))

print(len(train_0), len(test_0), len(test_1))

480 4320 4320


In [5]:
#converting lists to arrays and checking the shape

train_0 = np.array(train_0)
test_0 = np.array(test_0)
test_1 = np.array(test_1)
print(train_0.shape, test_0.shape, test_1.shape)

(480, 128, 128) (4320, 128, 128) (4320, 128, 128)


# Feature Extractors


### 1.Local Binary Patterns (LBP)

In [6]:
def compute_lbp(image, radius=2, n_points=16, method='uniform'):
  """
  Compute Local Binary Pattern (LBP) of an image.

  Args:
    image: Grayscale input image.
    radius: Radius of the circular neighborhood.
    n_points: Number of points in the neighborhood.
    method: Method to compute LBP ('uniform' or 'default').

  Returns:
    LBP histogram (feature vector).
  """
  lbp = local_binary_pattern(image, n_points, radius, method)
  # Histogram of LBP
  n_bins = int(lbp.max() + 1)
  hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins))
  hist = hist / hist.sum()  # Normalize histogram
  hist = np.array(hist)
  return hist

### 2. GLCM (Gray-Level Co-occurrence Matrix)

In [10]:
# Compute GLCM and extract features
def compute_glcm_features(gray_image, distances, angles):
  glcm = graycomatrix(
    gray_image,
    distances=distances,
    angles=angles,
    levels=256,  # 8-bit image has 256 levels
    symmetric=True,
    normed=True
  )

  # Extract features
  contrast = graycoprops(glcm, 'contrast')
  dissimilarity = graycoprops(glcm, 'dissimilarity')
  homogeneity = graycoprops(glcm, 'homogeneity')
  energy = graycoprops(glcm, 'energy')
  correlation = graycoprops(glcm, 'correlation')

  features = {
    'Contrast': contrast,
    'Correlation': correlation,
    'Energy': energy,
    'Homogeneity': homogeneity,
    'Dissimilarity': dissimilarity
  }

  feature_vector = []
  #how to initialize an empyt numpy array


  for key, value in features.items():
    for v in value:
      for k in v:
        feature_vector.append(k)

  feature_vector = np.array(feature_vector)

  return glcm, feature_vector


distances = [1, 2, 3]  # Pixel distances
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # Angles in radians

print(compute_glcm_features(train_0[0], distances, angles)[1])


[1.29297244e+02 2.00602393e+02 1.19780758e+02 2.51701593e+02
 3.52921193e+02 2.00602393e+02 3.33189608e+02 2.51701593e+02
 5.03768250e+02 4.72101600e+02 4.62945312e+02 5.53210066e+02
 8.38477600e-01 7.49415698e-01 8.50319813e-01 6.85574989e-01
 5.60106520e-01 7.49415698e-01 5.84182054e-01 6.85574989e-01
 3.72317450e-01 4.12329148e-01 4.23398444e-01 3.11352291e-01
 2.16116038e-02 1.94366186e-02 2.18652359e-02 1.87320424e-02
 1.77923841e-02 1.94366186e-02 1.78587575e-02 1.87320424e-02
 1.69425730e-02 1.69218481e-02 1.69893694e-02 1.66313788e-02
 1.02688617e-01 7.98704550e-02 1.10413555e-01 7.68038696e-02
 6.63750650e-02 7.98704550e-02 6.86084034e-02 7.68038696e-02
 5.57158519e-02 5.45557389e-02 5.84547817e-02 5.19947098e-02
 9.05351870e+00 1.14906070e+01 8.69525098e+00 1.27577035e+01
 1.49519469e+01 1.14906070e+01 1.45932540e+01 1.27577035e+01
 1.78196250e+01 1.75738851e+01 1.72329375e+01 1.89007937e+01]


### 3. FFT Features of the Texture Image

In [12]:
def compute_fft_features(image, num_rings=5, num_wedges=8):
  # Compute FFT and shift zero frequency to center
  fft = np.fft.fft2(image)
  fft_shifted = np.fft.fftshift(fft)
  magnitude_spectrum = np.abs(fft_shifted)

  # Get image dimensions and center
  h, w = image.shape
  cy, cx = h // 2, w // 2
  max_radius = int(np.sqrt(cy**2 + cx**2))

  # Create radius and angle maps
  y, x = np.ogrid[:h, :w]
  radius = np.sqrt((y - cy)**2 + (x - cx)**2)
  angle = np.arctan2(y - cy, x - cx) % (2 * np.pi)

  # Compute ring features
  ring_edges = np.linspace(0, max_radius, num_rings + 1)
  ring_features = [
    magnitude_spectrum[(radius >= ring_edges[i]) & (radius < ring_edges[i + 1])].sum()
    for i in range(num_rings)
  ]

  # Compute wedge features
  wedge_edges = np.linspace(0, 2 * np.pi, num_wedges + 1)
  wedge_features = [
    magnitude_spectrum[(angle >= wedge_edges[i]) & (angle < wedge_edges[i + 1])].sum()
    for i in range(num_wedges)
  ]

  # Flatten features by concatenating ring and wedge features
  flattened_features = np.array(ring_features + wedge_features)

  return flattened_features

image = train_0[0]

# Extract FFT features
num_rings = 5  # Number of rings
num_wedges = 8  # Number of wedges
fft_features = compute_fft_features(image, num_rings, num_wedges)

# Visualize features
print("FFT Features:", fft_features)


FFT Features: [7528569.57085799 7764039.4722744  3294343.46178634 1309487.60913622
  155390.19115493 4318971.59726571 1880077.63356567 2656243.32990535
 2206772.5047341  2233002.00015262 1890703.22069978 2670094.91703946
 2195994.10184718]


### 4. Gabor Features

In [13]:
def extract_gabor_features(image, ksize=25, sigma=5.0, lambd=5.0, gamma=0.5, psi=0, orientations=None):
    """
    Extract flattened Gabor features from an image using multiple orientations.

    Args:
        image_path (str): Path to the grayscale image.
        ksize (int): Size of the Gabor filter kernel.
        sigma (float): Standard deviation of the Gaussian envelope.
        lambd (float): Wavelength of the sinusoidal factor.
        gamma (float): Aspect ratio.
        psi (float): Phase offset.
        orientations (list): List of orientations in radians. Defaults to [0, π/4, π/2, 3π/4].

    Returns:
        flattened_features (np.ndarray): Flattened feature vector (mean and std for each orientation).
        filtered_images (list): List of filtered images for each orientation.
        kernels (list): List of Gabor filter kernels for each orientation.
    """
    if orientations is None:
        orientations = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]  # Default orientations

    # Lists to store results
    filtered_images = []
    kernels = []
    gabor_features = []

    # Apply Gabor filters for each orientation
    for theta in orientations:
        # Create Gabor kernel
        kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
        filtered = cv2.filter2D(image, cv2.CV_8UC3, kernel)
        mean = filtered.mean()
        std = filtered.std()
        gabor_features.append(mean)
        gabor_features.append(std)
        filtered_images.append(filtered)
        kernels.append(kernel)

    gabor_features = np.array(gabor_features)

    return gabor_features, filtered_images, kernel


# Create Feature Vectors

In [15]:
# Initialize dictionaries to store feature vectors for training and test sets
train = {}
test_0_dict = {}
test_1_dict = {}

# Define distances and angles for GLCM computation
distances = [1, 2, 3]  # Pixel distances
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # Angles in radians

# Initialize lists to store feature vectors
lbp = []
glcm = []
fft = []
gabor = []

# Compute features for training images
for image in train_0:
    lbp.append(compute_lbp(image))  # Compute LBP features
    _, image_features = compute_glcm_features(image, distances, angles)
    glcm.append(image_features)  # Compute GLCM features
    fft.append(compute_fft_features(image))  # Compute FFT features
    flattened_features, _, _ = extract_gabor_features(image)  # Compute Gabor features
    gabor.append(flattened_features)  # Compute Gabor features

# Store computed features in the training dictionary
train["LBP"] = np.array(lbp)
train["GLCM"] = np.array(glcm)
train["FFT"] = np.array(fft)
train["Gabor"] = np.array(gabor)

# Reset feature lists for test_0
lbp0 = []
glcm0 = []
fft0 = []
gabor0 = []

# Compute features for test_0 images
for i, image in enumerate(test_0):
    lbp0.append(compute_lbp(image))  # Compute LBP features
    _, image_features = compute_glcm_features(image, distances, angles)
    glcm0.append(image_features)  # Compute GLCM features
    fft0.append(compute_fft_features(image))  # Compute FFT features
    flattened_features, _, _ = extract_gabor_features(image)  # Compute Gabor features
    gabor0.append(flattened_features)  # Compute Gabor features

# Store computed features in the test_0 dictionary
test_0_dict["LBP"] = np.array(lbp0)
test_0_dict["GLCM"] = np.array(glcm0)
test_0_dict["FFT"] = np.array(fft0)
test_0_dict["Gabor"] = np.array(gabor0)

# Reset feature lists for test_1
lbp1 = []
glcm1 = []
fft1 = []
gabor1 = []

# Compute features for test_1 images
for image in test_1:
    lbp1.append(compute_lbp(image))  # Compute LBP features
    _, image_features = compute_glcm_features(image, distances, angles)
    glcm1.append(image_features)  # Compute GLCM features
    fft1.append(compute_fft_features(image))  # Compute FFT features
    flattened_features, _, _ = extract_gabor_features(image)  # Compute Gabor features
    gabor1.append(flattened_features)  # Compute Gabor features

# Store computed features in the test_1 dictionary
test_1_dict["LBP"] = np.array(lbp1)
test_1_dict["GLCM"] = np.array(glcm1)
test_1_dict["FFT"] = np.array(fft1)
test_1_dict["Gabor"] = np.array(gabor1)


print("Training set:")
print("LBP features:", train["LBP"])
print("GLCM features:", train["GLCM"])
print("FFT features:", train["FFT"])
print("Gabor features:", train["Gabor"])

print("\nTest set 0:")
print("LBP features:", test_0_dict["LBP"])
print("GLCM features:", test_0_dict["GLCM"])
print("FFT features:", test_0_dict["FFT"])
print("Gabor features:", test_0_dict["Gabor"])

print("\nTest set 1:")

print("LBP features:", test_1_dict["LBP"])
print("GLCM features:", test_1_dict["GLCM"])
print("FFT features:", test_1_dict["FFT"])
print("Gabor features:", test_1_dict["Gabor"])

print("\nShapes:")
print(f"Training set: {train['LBP'].shape}, {train['GLCM'].shape}, {train['FFT'].shape}, {train['Gabor'].shape}")
print(f"Test set 0: {test_0_dict['LBP'].shape}, {test_0_dict['GLCM'].shape}, {test_0_dict['FFT'].shape}, {test_0_dict['Gabor'].shape}")
print(f"Test set 1: {test_1_dict['LBP'].shape}, {test_1_dict['GLCM'].shape}, {test_1_dict['FFT'].shape}, {test_1_dict['Gabor'].shape}")

Training set:
LBP features: [[0.0390625  0.06884766 0.07946777 ... 0.05950928 0.04534912 0.08459473]
 [0.03656006 0.06652832 0.07897949 ... 0.05786133 0.04614258 0.08312988]
 [0.03796387 0.06463623 0.07635498 ... 0.06256104 0.04608154 0.08032227]
 ...
 [0.02905273 0.05853271 0.05255127 ... 0.05548096 0.05786133 0.09222412]
 [0.02990723 0.05194092 0.05706787 ... 0.05108643 0.04754639 0.0880127 ]
 [0.02862549 0.05462646 0.05834961 ... 0.05310059 0.05224609 0.08618164]]
GLCM features: [[1.29297244e+02 8.38477600e-01 2.16116038e-02 1.02688617e-01
  9.05351870e+00]
 [1.31487574e+02 8.36293752e-01 2.21286177e-02 1.09489919e-01
  9.08882874e+00]
 [1.58203740e+02 8.03225950e-01 2.29071629e-02 9.44713035e-02
  1.00690207e+01]
 ...
 [4.92589813e+01 9.38525276e-01 4.04948876e-02 2.16220782e-01
  5.10433071e+00]
 [6.67017101e+01 9.16687417e-01 4.29720194e-02 1.78639433e-01
  6.13096703e+00]
 [6.54130782e+01 9.18076357e-01 4.54018728e-02 1.94486012e-01
  5.94408219e+00]]
FFT features: [[7528569.570

In [62]:
print(train["LBP"].shape, test_0_dict["GLCM"].shape, test_1_dict["FFT"].shape)

(480, 10) (480, 5) (480, 13)


# Distance Functions

In [36]:
def euclidean_distance(x, y):
    """
    Compute the Euclidean distance between two feature vectors.

    Args:
        x: Feature vector 1 (1D array).
        y: Feature vector 2 (1D array).

    Returns:
        Euclidean distance (float).
    """
    return np.sqrt(np.sum((x - y) ** 2))


def manhattan_distance(x, y):
    """
    Compute the Manhattan distance between two feature vectors.

    Args:
        x: Feature vector 1 (1D array).
        y: Feature vector 2 (1D array).

    Returns:
        Manhattan distance (float).
    """
    return np.sum(np.abs(x - y))


def chi_squared_distance(x, y, epsilon=1e-10):
    """
    Compute the Chi-squared distance between two feature vectors.

    Args:
        x: Feature vector 1 (1D array).
        y: Feature vector 2 (1D array).
        epsilon: Small constant to avoid division by zero (default: 1e-10).

    Returns:
        Chi-squared distance (float).
    """
    return np.sum((x - y) ** 2 / (x + y + epsilon))


def compute_inverse_covariance_matrix(train_features):
    """
    Compute the regularized inverse covariance matrix.

    Args:
        train_features (np.ndarray): Feature vectors of training data.

    Returns:
        inv_cov_matrix (np.ndarray): Regularized inverse covariance matrix.
    """
    # Compute the covariance matrix
    cov_matrix = np.cov(train_features, rowvar=False)
    
    # Add a small regularization term to the diagonal
    regularization_term = 1e-5  # Adjust this value if needed
    cov_matrix += np.eye(cov_matrix.shape[0]) * regularization_term
    
    # Compute the inverse
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    return inv_cov_matrix



def mahalanobis_distance(x, y, inv_cov_matrix):
    """
    Compute the Mahalanobis distance between two feature vectors.

    Args:
        x: Feature vector 1 (1D array).
        y: Feature vector 2 (1D array).
        inv_cov_matrix: Inverse covariance matrix of the training data.

    Returns:
        Mahalanobis distance (float).
    """
    diff = x - y
    return np.sqrt(np.dot(np.dot(diff.T, inv_cov_matrix), diff))






# Nearest Neighbour Algorithm

In [34]:
def nearest_neighbour(train_feature_vector : np.ndarray, train_labels : np.ndarray, test_feature_vectors : np.ndarray, metric = "Euclidean"):
    """
        Perform Nearest Neighbor classification with k=1.

        Args:
            train_feature_vector: Feature vectors of training samples (2D array).
            train_labels: Labels of training samples (1D array).
            test_feature_vectors: Feature vectors of test samples (2D array).
            metric: Distance metric to use ('euclidean', 'manhattan', 'chi_squared', 'mahalanobis').

        Returns:
            Predicted labels for the test samples.
    """

    test_x_len, _ = test_feature_vectors.shape
    train_x_len, _ = train_feature_vector.shape

    y_predict = np.zeros(test_x_len, dtype=int)

    if metric == "Euclidean":
        
        for i in range(test_x_len):

            x_test = test_feature_vectors[i]

            neighbours = np.zeros(1)

            euclidean_distances = np.zeros(train_x_len, np.float32)

            for j in range(train_x_len):
                
                distance = euclidean_distance(x_test, train_feature_vector[j])

                euclidean_distances[j] = distance

            inds = euclidean_distances.argsort() 
          
            Y_train_sorted = train_labels[inds] 

            neighbours = Y_train_sorted[0]


            y_predict[i] = neighbours


    
    elif metric == "Manhattan":
        for i in range(test_x_len):

            x_test = test_feature_vectors[i]

            neighbours = np.zeros(1)

            manhattan_distances = np.zeros(train_x_len, np.float32)

            for j in range(train_x_len):
                
                distance = manhattan_distance(x_test, train_feature_vector[j])

                manhattan_distances[j] = distance

            inds = manhattan_distances.argsort() 
          
            Y_train_sorted = train_labels[inds] 

            neighbours = Y_train_sorted[0]


            y_predict[i] = neighbours

    elif metric == "Chi-Squared":
         for i in range(test_x_len):

            x_test = test_feature_vectors[i]

            neighbours = np.zeros(1)

            chi_squared_distances = np.zeros(train_x_len, np.float32)

            for j in range(train_x_len):
                
                distance = chi_squared_distance(x_test, train_feature_vector[j])

                chi_squared_distances[j] = distance

            inds = chi_squared_distances.argsort() 
          
            Y_train_sorted = train_labels[inds] 

            neighbours = Y_train_sorted[0]


            y_predict[i] = neighbours

    elif metric == "Mahalanobis":
        
        inv_cov_matrix = compute_inverse_covariance_matrix(train_feature_vector)

        for i in range(test_x_len):

            x_test = test_feature_vectors[i]

            neighbours = np.zeros(1)

            mahalanobis_distances = np.zeros(train_x_len, np.float32)

            for j in range(train_x_len):
                
                distance = mahalanobis_distance(x_test, train_feature_vector[j], inv_cov_matrix)

                mahalanobis_distances[j] = distance

            inds = mahalanobis_distances.argsort() 
          
            Y_train_sorted = train_labels[inds] 

            neighbours = Y_train_sorted[0]

            y_predict[i] = neighbours
    
    else:
        raise Exception(f"{metric} distance metric is not supported by this implementation")
    
    return y_predict

In [16]:
# labels

train_labels = []

with open('imageset/000/train.txt') as f:
    labels = f.readlines()
    for line in labels[1:]:
        _, label = line.split()
        label = label.strip()
        train_labels.append(int(label))

train_labels = np.array(train_labels)


test0_labels = []

with open('test1.txt') as f:
    labels = f.readlines()
    for line in labels[1:]:
        _, label = line.split()
        label = label.strip()
        test0_labels.append(int(label))

test0_labels = np.array(test0_labels)


test1_labels = []

with open('test2.txt') as f:
    labels = f.readlines()
    for line in labels[1:]:
        _, label = line.split()
        label = label.strip()
        test1_labels.append(int(label))

test1_labels = np.array(test1_labels)

print(test0_labels)

[ 0  0  0 ... 23 23 23]


# Implementation

## Evaluation Function

In [18]:
from sklearn.metrics import accuracy_score

def evaluate_classification(true_labels, predicted_labels):
    """
    Evaluate classification accuracy.
    
    Args:
        true_labels: Ground truth labels.
        predicted_labels: Predicted labels.

    Returns:
        Accuracy score.
    """
    return accuracy_score(true_labels, predicted_labels)

## All metrics with all features

In [32]:
def calculate_results(train, train_labels, test_0_dict, test_0_labels, test_1_dict, test_1_labels):

    distance_metrics = ["Euclidean", "Manhattan", "Chi-Squared", "Mahalanobis"]
    feature_types = ["LBP", "GLCM", "FFT", "Gabor"]

    results = {}

    for feature in feature_types:

        for metric in distance_metrics:
            test0_predict = nearest_neighbour(train[feature], train_labels, test_0_dict[feature], metric=metric)
            test1_predict = nearest_neighbour(train[feature], train_labels, test_1_dict[feature], metric=metric)

            accuracy_test0 = evaluate_classification(test0_predict, test_0_labels)
            accuracy_test1 = evaluate_classification(test1_predict, test_1_labels)

            results[f"{feature} - {metric}"] = {
                "Test 0 Accuracy": accuracy_test0,
                "Test 1 Accuracy": accuracy_test1
            }
    results = pd.DataFrame(results).T

    return results



In [56]:
initial_result = pd.DataFrame(results).T
initial_result

,Test 0 Accuracy,Test 1 Accuracy
LBP - Euclidean,0.639352,0.618981
LBP - Manhattan,0.642593,0.605093
LBP - Chi-Squared,0.665278,0.652083
LBP - Mahalanobis,0.695833,0.710185
GLCM - Euclidean,0.240046,0.219444
GLCM - Manhattan,0.255324,0.236806
GLCM - Chi-Squared,0.357870,0.340741
GLCM - Mahalanobis,0.447685,0.499537
FFT - Euclidean,0.647917,0.614352
FFT - Manhattan,0.644213,0.617824


## Combinations

In [73]:
def calculate_combinations_result(train, train_labels, test_0_dict, test_0_labels, test_1_dict, test_1_labels):

    distance_metrics = ["Euclidean", "Manhattan", "Chi-Squared", "Mahalanobis"]
    feature_types = ["LBP", "GLCM", "FFT", "Gabor"]

    combination_results = {}

    for feature1 in feature_types:
        for feature2 in feature_types:
            for metric in distance_metrics:

                combined_train_features = np.concatenate((train[feature1], train[feature2]), axis=1)
                combined_test0_features = np.concatenate((test_0_dict[feature1], test_0_dict[feature2]), axis=1)
                combined_test1_features = np.concatenate((test_1_dict[feature1], test_1_dict[feature2]), axis=1)

                test0_predict = nearest_neighbour(combined_train_features, train_labels, combined_test0_features, metric=metric)
                test1_predict = nearest_neighbour(combined_train_features, train_labels, combined_test1_features, metric=metric)

                accuracy_test0 = evaluate_classification(test0_predict, test_0_labels)
                accuracy_test1 = evaluate_classification(test1_predict, test_1_labels)

                combination_results[f"{feature1}"] = {"feature2": feature2, "metric" : metric, "Test 0 Accuracy": accuracy_test0, "Test 1 Accuracy": accuracy_test1}

    combination_results = pd.DataFrame(combination_results).T
    return combination_results 

In [38]:
from sklearn.neighbors import KNeighborsClassifier


def calculate_combinations_result_scikit(train, train_labels, test_0_dict, test_0_labels, test_1_dict, test_1_labels):

    distance_metrics = ["euclidean", "manhattan", "mahalanobis"]
    feature_types = ["LBP", "GLCM", "FFT", "Gabor"]

    combination_results = {}

    for feature1 in feature_types:
        for feature2 in feature_types:
            if feature1 != feature2:

                combined_train_features = np.concatenate((train[feature1], train[feature2]), axis=1)
                combined_test0_features = np.concatenate((test_0_dict[feature1], test_0_dict[feature2]), axis=1)
                combined_test1_features = np.concatenate((test_1_dict[feature1], test_1_dict[feature2]), axis=1)
                
                for metric in distance_metrics:
                    if metric == "mahalanobis":
                        model1 = KNeighborsClassifier(n_neighbors=1, metric=metric, metric_params={'VI': np.cov(combined_train_features.T)})
                    else:
                        model1 = KNeighborsClassifier(n_neighbors=1, metric=metric)
                    model1.fit(combined_train_features, train_labels)
                    test0_predict = model1.predict(combined_test0_features)
                    test1_predict = model1.predict(combined_test1_features)

                    """test0_predict = nearest_neighbour(combined_train_features, train_labels, combined_test0_features, metric=metric)
                    test1_predict = nearest_neighbour(combined_train_features, train_labels, combined_test1_features, metric=metric)"""

                    accuracy_test0 = evaluate_classification(test0_predict, test_0_labels)
                    accuracy_test1 = evaluate_classification(test1_predict, test_1_labels)

                    combination_results[f"{feature1} - {feature2} - {metric}"] = {"Test 0 Accuracy": accuracy_test0, "Test 1 Accuracy": accuracy_test1}
            print(1)

    print(combination_results)

    combination_results = pd.DataFrame(combination_results).T
    return combination_results 

In [58]:
combination_results = pd.DataFrame(combination_results).T
combination_results 

,feature2,metric,Test 0 Accuracy,Test 1 Accuracy
LBP,Gabor,Mahalanobis,0.515972,0.543287
GLCM,Gabor,Mahalanobis,0.399074,0.425926
FFT,FFT,Chi-Squared,0.617361,0.580093


In [ ]:
result= nearest_neighbour(train["Gabor"], train_labels, test_0_dict["Gabor"], metric="Euclidean")
print(evaluate_classification(test0_labels, result))



In [48]:
comb = np.concatenate((train["LBP"], train["FFT"]), axis=1)
print(comb.shape)
test0_comb = np.concatenate((test_0_dict["LBP"], test_0_dict["FFT"]), axis=1)
print(test0_comb.shape)

(480, 23)
(4320, 23)


In [51]:
result_comb = nearest_neighbour(comb, train_labels, test0_comb, metric="Manhattan")
print(evaluate_classification(test0_labels, result_comb))

0.6442129629629629


In [ ]:
for key, value in results.items():
    print(key, value)

In [99]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=1, metric='mahalanobis', metric_params={'VI': np.cov(comb.T)})

model.fit(comb, train_labels)

res = model.predict(test0_comb)


print(evaluate_classification(test0_labels, res))

0.6479166666666667


In [10]:
import cv2
import numpy as np

def gabor_larger_image_bank(image, ksize_list=[7, 11, 15, 21], sigma_=[2.0, 3.0, 4.0], 
                            lambd_=[4, 8, 16], gamma=0.5, psi=0, orientations=None):
    """
    Extract flattened Gabor features from an image using multiple kernel sizes, orientations, sigma, and wavelengths.

    Args:
        image (np.ndarray): Grayscale input image.
        ksize_list (list): List of kernel sizes for the Gabor filter.
        sigma_ (list): List of standard deviations of the Gaussian envelope.
        lambd_ (list): List of wavelengths of the sinusoidal factor.
        gamma (float): Aspect ratio.
        psi (float): Phase offset.
        orientations (list): List of orientations in radians. Defaults to [0, π/4, π/2, 3π/4].

    Returns:
        gabor_features (np.ndarray): Flattened feature vector (mean and std for each configuration).
        filtered_images (list): List of filtered images for each configuration.
        kernels (list): List of Gabor filter kernels for each configuration.
    """
    if orientations is None:
        orientations = [0, np.pi / 8, np.pi / 4, 3 * np.pi / 8, np.pi / 2, 
                        5 * np.pi / 8, 3 * np.pi / 4, 7 * np.pi / 8]  # Default orientations

    # Lists to store results
    filtered_images = []
    kernels = []
    gabor_features = []

    # Iterate through all combinations of kernel sizes, sigma, lambd, and orientations
    for ksize in ksize_list:
        for theta in orientations:
            for sigma in sigma_:
                for lambd in lambd_:
                    # Create Gabor kernel
                    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
                    
                    # Apply Gabor filter to the image
                    filtered = cv2.filter2D(image, cv2.CV_8UC3, kernel)
                    
                    # Calculate mean and standard deviation as features
                    mean = filtered.mean()
                    std = filtered.std()
                    gabor_features.append(mean)
                    gabor_features.append(std)
                    
                    # Store the filtered image and kernel
                    filtered_images.append(filtered)
                    kernels.append(kernel)

    # Flatten the feature vector
    gabor_features = np.array(gabor_features)

    return gabor_features, filtered_images, kernels


In [11]:
new_gabor_train = []
new_gabor_test0 = []
new_gabor_test1 = []


for image in train_0:
    flattened_features, _, _ = gabor_larger_image_bank(image)
    new_gabor_train.append(flattened_features)

for image in test_0:
    flattened_features, _, _ = gabor_larger_image_bank(image)
    new_gabor_test0.append(flattened_features)

for image in test_1:
    flattened_features, _, _ = gabor_larger_image_bank(image)
    new_gabor_test1.append(flattened_features)

new_gabor_train = np.array(new_gabor_train)
new_gabor_test0 = np.array(new_gabor_test0)
new_gabor_test1 = np.array(new_gabor_test1)



KeyboardInterrupt: 

In [63]:
new_Result = nearest_neighbour(new_gabor_train, train_labels, new_gabor_test0, metric="Euclidean")
print(evaluate_classification(test0_labels, new_Result))

0.47939814814814813


In [69]:
print(new_gabor_test0.shape)

(4320, 576)


In [12]:
# Initialize dictionaries to store feature vectors for training and test sets
train_2 = {}
test_0_dict_2 = {}
test_1_dict_2 = {}

# Define distances and angles for GLCM computation
distances = [1, 2, 3, 5]  # Fine and coarse distances
angles = [0, np.pi/8, np.pi/4, 3*np.pi/8, np.pi/2, 5*np.pi/8, 3*np.pi/4, 7*np.pi/8]

# Initialize lists to store feature vectors
lbp_ = []
glcm_ = []
fft_ = []
gabor_ = []

# Compute features for training images
for image in train_0:
    lbp_.append(compute_lbp(image))  # Compute LBP features
    _, image_features = compute_glcm_features(image, distances, angles)
    glcm_.append(image_features)  # Compute GLCM features
    fft_.append(compute_fft_features(image, 8, 12))  # Compute FFT features
    flattened_features, _, _ = gabor_larger_image_bank(image)  # Compute Gabor features
    gabor_.append(flattened_features)  # Compute Gabor features

# Store computed features in the training dictionary
train_2["LBP"] = np.array(lbp_)
train_2["GLCM"] = np.array(glcm_)
train_2["FFT"] = np.array(fft_)
train_2["Gabor"] = np.array(gabor_)

# Reset feature lists for test_0
lbp0_ = []
glcm0_ = []
fft0_ = []
gabor0_ = []

# Compute features for test_0 images
for i, image in enumerate(test_0):
    lbp0_.append(compute_lbp(image))  # Compute LBP features
    _, image_features = compute_glcm_features(image, distances, angles)
    glcm0_.append(image_features)  # Compute GLCM features
    fft0_.append(compute_fft_features(image, 8, 12))  # Compute FFT features
    flattened_features, _, _ = gabor_larger_image_bank(image)  # Compute Gabor features
    gabor0_.append(flattened_features)  # Compute Gabor features

# Store computed features in the test_0 dictionary
test_0_dict_2["LBP"] = np.array(lbp0_)
test_0_dict_2["GLCM"] = np.array(glcm0_)
test_0_dict_2["FFT"] = np.array(fft0_)
test_0_dict_2["Gabor"] = np.array(gabor0_)

# Reset feature lists for test_1
lbp1_ = []
glcm1_ = []
fft1_ = []
gabor1_ = []

# Compute features for test_1 images
for image in test_1:
    lbp1_.append(compute_lbp(image))  # Compute LBP features
    _, image_features = compute_glcm_features(image, distances, angles)
    glcm1_.append(image_features)  # Compute GLCM features
    fft1_.append(compute_fft_features(image, 8, 12))  # Compute FFT features
    flattened_features, _, _ = gabor_larger_image_bank(image)  # Compute Gabor features
    gabor1_.append(flattened_features)  # Compute Gabor features

# Store computed features in the test_1 dictionary
test_1_dict_2["LBP"] = np.array(lbp1_)
test_1_dict_2["GLCM"] = np.array(glcm1_)
test_1_dict_2["FFT"] = np.array(fft1_)
test_1_dict_2["Gabor"] = np.array(gabor1_)



In [14]:
print("\nShapes:")
print(f"Training set: {train_2['LBP'].shape}, {train_2['GLCM'].shape}, {train_2['FFT'].shape}, {train_2['Gabor'].shape}")
print(f"Test set 0: {test_0_dict_2['LBP'].shape}, {test_0_dict_2['GLCM'].shape}, {test_0_dict_2['FFT'].shape}, {test_0_dict_2['Gabor'].shape}")
print(f"Test set 1: {test_1_dict_2['LBP'].shape}, {test_1_dict_2['GLCM'].shape}, {test_1_dict_2['FFT'].shape}, {test_1_dict_2['Gabor'].shape}")


Shapes:
Training set: (480, 18), (480, 5), (480, 20), (480, 576)
Test set 0: (4320, 18), (4320, 5), (4320, 20), (4320, 576)
Test set 1: (4320, 18), (4320, 5), (4320, 20), (4320, 576)


In [77]:
second_results = calculate_results(train_2, train_labels, test_0_dict_2, test0_labels, test_1_dict_2, test1_labels)
second_results

,Test 0 Accuracy,Test 1 Accuracy
LBP - Euclidean,0.639352,0.618981
LBP - Manhattan,0.642593,0.605093
LBP - Chi-Squared,0.665278,0.652083
LBP - Mahalanobis,0.695833,0.710185
GLCM - Euclidean,0.240046,0.219444
GLCM - Manhattan,0.255324,0.236806
GLCM - Chi-Squared,0.357870,0.340741
GLCM - Mahalanobis,0.447685,0.499537
FFT - Euclidean,0.686574,0.679630
FFT - Manhattan,0.686806,0.683565


In [39]:
second_comb_result = calculate_combinations_result_scikit(train_2, train_labels, test_0_dict_2, test0_labels, test_1_dict_2, test1_labels)
second_comb_result

1
1


KeyboardInterrupt: 

In [20]:
second_comb_result

,feature2,metric,Test 0 Accuracy,Test 1 Accuracy
LBP,Gabor,mahalanobis,0.446528,0.454167
GLCM,Gabor,mahalanobis,0.440509,0.435185
FFT,Gabor,mahalanobis,0.672454,0.619907
Gabor,FFT,mahalanobis,0.672454,0.619907


# Counter Measures Against Illumination Variations

In [27]:
def normalize_equalize(image):
    """
    Normalize and equalize the input image.

    Args:
        image (np.ndarray): Grayscale input image.

    Returns:
        Normalized and equalized image.
    """
    # Normalize the image
    normalized = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    # Equalize the histogram
    equalized = cv2.equalizeHist(normalized)
    
    return equalized

norm_train = []

norm_test_0 = []

norm_test_1 = []

for image in train_0:
    norm_train.append(normalize_equalize(image))

for image in test_0:
    norm_test_0.append(normalize_equalize(image))

for image in test_1:
    norm_test_1.append(normalize_equalize(image))

norm_train = np.array(norm_train)
norm_test_0 = np.array(norm_test_0)
norm_test_1 = np.array(norm_test_1)

print(norm_train.shape, norm_test_0.shape, norm_test_1.shape)


(480, 128, 128) (4320, 128, 128) (4320, 128, 128)


In [28]:
def compute_vectors(train, test_0, test_1):
    # Initialize dictionaries to store feature vectors for training and test sets
    train_vector = {}
    test_0_vector = {}
    test_1_vector = {}

    # Define distances and angles for GLCM computation
    distances = [1, 2, 3, 5]  # Fine and coarse distances
    angles = [0, np.pi/8, np.pi/4, 3*np.pi/8, np.pi/2, 5*np.pi/8, 3*np.pi/4, 7*np.pi/8]

    # Initialize lists to store feature vectors
    lbp_ = []
    glcm_ = []
    fft_ = []
    gabor_ = []

    # Compute features for training images
    for image in train:
        lbp_.append(compute_lbp(image))  # Compute LBP features
        _, image_features = compute_glcm_features(image, distances, angles)
        glcm_.append(image_features)  # Compute GLCM features
        fft_.append(compute_fft_features(image, 8, 12))  # Compute FFT features
        flattened_features, _, _ = gabor_larger_image_bank(image)  # Compute Gabor features
        gabor_.append(flattened_features)  # Compute Gabor features

    # Store computed features in the training dictionary
    train_vector["LBP"] = np.array(lbp_)
    train_vector["GLCM"] = np.array(glcm_)
    train_vector["FFT"] = np.array(fft_)
    train_vector["Gabor"] = np.array(gabor_)

    # Reset feature lists for test_0
    lbp0_ = []
    glcm0_ = []
    fft0_ = []
    gabor0_ = []

    # Compute features for test_0 images
    for i, image in enumerate(test_0):
        lbp0_.append(compute_lbp(image))  # Compute LBP features
        _, image_features = compute_glcm_features(image, distances, angles)
        glcm0_.append(image_features)  # Compute GLCM features
        fft0_.append(compute_fft_features(image, 8, 12))  # Compute FFT features
        flattened_features, _, _ = gabor_larger_image_bank(image)  # Compute Gabor features
        gabor0_.append(flattened_features)  # Compute Gabor features

    # Store computed features in the test_0 dictionary
    test_0_vector["LBP"] = np.array(lbp0_)
    test_0_vector["GLCM"] = np.array(glcm0_)
    test_0_vector["FFT"] = np.array(fft0_)
    test_0_vector["Gabor"] = np.array(gabor0_)

    # Reset feature lists for test_1
    lbp1_ = []
    glcm1_ = []
    fft1_ = []
    gabor1_ = []

    # Compute features for test_1 images
    for image in test_1:
        lbp1_.append(compute_lbp(image))  # Compute LBP features
        _, image_features = compute_glcm_features(image, distances, angles)
        glcm1_.append(image_features)  # Compute GLCM features
        fft1_.append(compute_fft_features(image, 8, 12))  # Compute FFT features
        flattened_features, _, _ = gabor_larger_image_bank(image)  # Compute Gabor features
        gabor1_.append(flattened_features)  # Compute Gabor features

    # Store computed features in the test_1 dictionary
    test_1_vector["LBP"] = np.array(lbp1_)
    test_1_vector["GLCM"] = np.array(glcm1_)
    test_1_vector["FFT"] = np.array(fft1_)
    test_1_vector["Gabor"] = np.array(gabor1_)

    return train_vector, test_0_vector, test_1_vector


In [ ]:
train_vector, test_0_vector, test_1_vector = compute_vectors(norm_train, norm_test_0, norm_test_1)



NameError: name 'calculate_results' is not defined

In [37]:
normalized_results = calculate_results(train_vector, train_labels, test_0_vector, test0_labels, test_1_vector, test1_labels)

normalized_results 

,Test 0 Accuracy,Test 1 Accuracy
LBP - Euclidean,0.796991,0.708796
LBP - Manhattan,0.801157,0.724537
LBP - Chi-Squared,0.813889,0.757176
LBP - Mahalanobis,0.787269,0.815741
GLCM - Euclidean,0.172685,0.153704
GLCM - Manhattan,0.182639,0.163889
GLCM - Chi-Squared,0.258333,0.233102
GLCM - Mahalanobis,0.488657,0.503935
FFT - Euclidean,0.692593,0.673380
FFT - Manhattan,0.678241,0.662037
